# Using DFTK on GPUs

In this example we will look how DFTK can be used on
Graphics Processing Units.
In its current state runs based on Nvidia GPUs
using the [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl) Julia
package are better supported and there are considerably less rough
edges.

> **GPU parallelism not supported everywhere**
>
> GPU support is still a relatively new feature in DFTK.
> While basic SCF computations and e.g. forces are supported,
> this is not yet the case for all parts of the code.
> In most cases there is no intrinsic limitation and typically it only takes
> minor code modification to make it work on GPUs.
> If you require GPU support in one of our routines, where this is not
> yet supported, feel free to open an issue on github or otherwise get in touch.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

**Model setup.** First step is to setup a `Model` in DFTK.
This proceeds exactly as in the standard CPU case
(see also our Tutorial).

In [2]:
silicon = bulk(:Si)

model  = model_DFT(silicon;
                   functionals=PBE(),
                   pseudopotentials=PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf"))
nothing  # hide

Next is the selection of the computational architecture.
This effectively makes the choice, whether the computation will be run
on the CPU or on a GPU.

**Nvidia GPUs.**
Supported via [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl).
If you install the CUDA package, all required Nvidia cuda libraries
will be automatically downloaded. So literally, the only thing
you have to do is:

In [3]:
using CUDA
architecture = DFTK.GPU(CuArray)

DFTK.GPU{CUDA.CuArray}()

**AMD GPUs.** Supported via [AMDGPU.jl](https://github.com/JuliaGPU/AMDGPU.jl).
Here you need to [install ROCm](https://rocm.docs.amd.com/) manually.
With that in place you can then select:

In [4]:
using AMDGPU
architecture = DFTK.GPU(ROCArray)

DFTK.GPU{AMDGPU.ROCArray}()

**Portable architecture selection.**
To make sure this script runs on the github CI (where we don't have GPUs
available) we check for the availability of GPUs before selecting an
architecture:

In [5]:
architecture = has_cuda() ? DFTK.GPU(CuArray) : DFTK.CPU()

DFTK.CPU()

**Basis and SCF.**
Based on the `architecture` we construct a `PlaneWaveBasis` object
as usual:

In [6]:
basis  = PlaneWaveBasis(model; Ecut=30, kgrid=(5, 5, 5), architecture)
nothing  # hide

... and run the SCF and some post-processing:

In [7]:
scfres = self_consistent_field(basis; tol=1e-6)
compute_forces(scfres)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.457757989297                   -0.94    5.4    10.0s
  2   -8.459893543928       -2.67       -1.78    1.0    4.25s
  3   -8.460053901322       -3.79       -2.91    1.9    427ms
  4   -8.460064826672       -4.96       -3.35    3.3    571ms
  5   -8.460064906727       -7.10       -3.88    1.3    354ms
  6   -8.460064914664       -8.10       -4.94    1.5    376ms
  7   -8.460064915250       -9.23       -5.21    3.1    551ms
  8   -8.460064915265      -10.82       -6.57    1.0    333ms


2-element Vector{StaticArraysCore.SVector{3, Float64}}:
 [-1.3365863875947015e-14, -1.513583889452179e-14, -1.875368152940989e-14]
 [1.503486850002228e-14, 1.6163199381392244e-14, 1.901399022937159e-14]

> **GPU performance**
>
> Our current (May 2025) benchmarks show DFTK to have reasonable performance
> on Nvidia / CUDA GPUs with up to a 100-fold speed-up over single-threaded
> CPU execution. However, support on AMD GPUs has been less benchmarked and
> there are likely rough edges. Since GPU support in DFTK is relatively new
> we appreciate any experience reports or bug reports.